In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings 
sns.set_style('darkgrid')
warnings.filterwarnings('ignore')
import re

from nltk.corpus import stopwords
from tqdm.auto import tqdm
tqdm.pandas()
import pymorphy2

m = pymorphy2.MorphAnalyzer()
import nltk

nltk.download('stopwords')
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim import similarities
from gensim.models import lsimodel
import math
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alimc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from pandas import datetime

In [3]:
! pip install pymorphy2

In [4]:
#Теги
accountant = r'(бухгалтер) | (бухгалтер) | (бухгалтерия) | (бух.*?учет) | (налог) | (налоги) | (налоговая) |\
(ЕГАИС) | (кадровый учет) | (ФСБУ 25/2018) | (НДС) | (инспекция) | (аккредитация) | (новый) |\
(изменения) | (изменение) | (ИП) | (ООО) | (деньги) | (денег) | (НДФЛ) | (ЭДО) | (банкрот) | (банкротство) |\
(счет) | (акт) | (оплата) | (зарплата) | (налогообложение) | (сделка) | (отчет) | (отчёт) | (долг) | (аванс) | (УПД) |\
(лицензия) | (работодатель) | (документы) | (справка) | (минфин) | (министерство финансов) | (МРОТ) | (выручка)|\
(НДС) | (накладная) | (страховка)|\
(кредит) | (кредитование) | (физлицо) | (юрлицо) | (финансы) | (финансирование) | (имущество) |\
(статформа) | (статформу) | (статформы) | (недвижимость) | (расходы) | (расходов) | (расходах)'  
              
business = r'(бизнес) | (бизнес) | (менеджмент) | (тайм-менеджмент) | (управление)|\
(управленец) | (новый) | (изменения) | (ИП) | (ООО) | (деньги) | (денег) | (самозанятый)|\
(самозанятость) | (банкрот) | (банкротство) | (товар) | (бизнес.*?стратегия) | (бизнес.*?идея)|\
(сделка) | (выручка) | (реклама) | (продукт) | (MVP) | (продажи) | (планирование) | (клиент) | (клиенты)|\
(конкурент) | (конкуренция) | (кризис) | (экспорт) | (импорт) | (рынок) | (доход) | (убыток) | (поставки)|\
(продажи) | (производитель) | (производство рост) | (снижение) | (спрос) | (предложение)|\
(мылый.*?бизнес) | (средний.*?бизнес) | (кредит) | (кредитование) | (физлицо) | (юрлицо) | (бизнеса)|\
(бизнесом) | (бизнесу) | (бизнесе) | (финансы) | (финансирование) | (расходы) | (расходов) | (расходах)'

In [5]:
print(business)

(бизнес) | (бизнес) | (менеджмент) | (тайм-менеджмент) | (управление)|\
(управленец) | (новый) | (изменения) | (ИП) | (ООО) | (деньги) | (денег) | (самозанятый)|\
(самозанятость) | (банкрот) | (банкротство) | (товар) | (бизнес.*?стратегия) | (бизнес.*?идея)|\
(сделка) | (выручка) | (реклама) | (продукт) | (MVP) | (продажи) | (планирование) | (клиент) | (клиенты)|\
(конкурент) | (конкуренция) | (кризис) | (экспорт) | (импорт) | (рынок) | (доход) | (убыток) | (поставки)|\
(продажи) | (производитель) | (производство рост) | (снижение) | (спрос) | (предложение)|\
(мылый.*?бизнес) | (средний.*?бизнес) | (кредит) | (кредитование) | (физлицо) | (юрлицо) | (бизнеса)|\
(бизнесом) | (бизнесу) | (бизнесе) | (финансы) | (финансирование) | (расходы) | (расходов) | (расходах)


In [6]:
#Преобразование слов в именительный падеж
mystopwords = stopwords.words('russian') + [
    'это', 'наш' , 'тыс', 'млн', 'млрд', 'также',  'т', 'д',
    'который','прошлый','сей', 'свой', 'наш', 'мочь', 'такой'
]
ru_words = re.compile("[А-Яа-я]+")


def words_only(text):
    return " ".join(ru_words.findall(text))


def lemmatize(text):
    try:
        return  " ".join([m.parse(w)[0].normal_form for w in text.lower().split()])
    except:
        return " "


def remove_stopwords(text, mystopwords = mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

    
def preprocess(text):
    return remove_stopwords(lemmatize(words_only(text.lower())))


In [7]:
class DataPreprocessor:
    def __init__(self):
        
        self.tags_find = None
        
        self.name_tags = None
        
        self.count_tags = None
        
        self.cl = 0
        
    def prime_fit(self, df, profession): 
        
        # Подсчет количества профессий, с возможностью расширения ролей 
        
        self.cl += 1 
        
        self.tags_find = df["union_im"].apply(lambda x: re.findall(profession, str(x).lower()))
        
        self.name_tags = 'tags_profession' + str(self.cl)

        self.count_tags = 'count_profession'+ str(self.cl)
        
#     def fit(self, df, profession, count): 
#         #Поиск тегов
        
#         self.tags_find = df["union_im"].apply(lambda x: re.findall(profession, str(x).lower()))
        
#         self.name_tags = 'tags_profession' + str(count)

#         self.count_tags = 'count_profession'+ str(count)
        
    def transform(self, df, profession):
        
        #Очистка тегов
        ans = []
        for line in (self.tags_find):
            buf = []
            if len(line):
                buf1 = ""
                for j in range(len(line)):    
                    el = set(line[j])
                    el.remove('')
                    element = str(el.pop())
                    if (element not in buf1):
                        buf1 += element + ' '         
                ans.append(buf1)
            else:
                ans.append(None)
                
        
        df[self.name_tags] = ans
       
        #Подсчет тегов
        
        df[self.count_tags] = 0
        for i, line in enumerate(df[self.name_tags]):
            if line != None:
                df[self.count_tags].iloc[i] = len(line.split())
            else:
                df[self.count_tags].iloc[i] = 0
                
        # Добавление нового датафрема к основному
        
        
        return df
    
#     def new_data_loading(self, path_data):
        
#         # загрузка нового дата фрема
        
#         df = pd.read_csv(path_data)
        
#         df.rename(columns = {"Title": "title", "Description": "text", "Links": "links", "Publication Date": "date"}, inplace = True)
        
#         # Обработка даты

#         df['date'] = df['date'].apply(lambda x: pd.Timestamp(x)) 
        
#         df = df.loc[df["text"].isna() == False]

        
#         # Приведение текста в именительный падеж
        
#         df["text_im"] = df["text"].astype('str').progress_apply(preprocess)
        
#         df["title_im"] = df["title"].astype('str').progress_apply(preprocess)
        
#         # объединение текста
        
#         df["union_im"] = df["text_im"] + df["title_im"]
        
#         # удаление ненужного
#         df.drop("text_im", axis = 1, inplace = True)
        
#         df.drop("title_im", axis = 1, inplace = True)
        
#         # Удаление пустых строк
#         df = df.loc[df["text"].isna() == False]
        
#         return df

    def prime_data_loading(self, path_data):
        
        prime_news = pd.read_csv(path_data)
        
        # Обработка даты
        prime_news['date'] = prime_news['date'].apply(lambda x: pd.Timestamp(x))
        
        
        # Удаление ненужного столбца 
        prime_news.drop("tags", axis = 1, inplace = True)
        
        # Приведение текста в именительный падеж
        prime_news["text_im"] = prime_news["text"].astype('str').progress_apply(preprocess)
        prime_news["title_im"] = prime_news["title"].astype('str').progress_apply(preprocess)
        
         # объединение текста
        prime_news["union_im"] = prime_news["text_im"] + prime_news["title_im"]
        
        # удаление ненужного
        prime_news.drop("text_im", axis = 1, inplace = True)
        
        prime_news.drop("title_im", axis = 1, inplace = True)
        prime_news.drop("Unnamed: 0", axis = 1, inplace = True)
        
        return prime_news
    
    def data_concat(self, prime_df, df):      
        
        #Добавление нового датафрема к основному
        prime_df = pd.concat([prime_df, df], axis=0, ignore_index=True)
        
        return prime_df
    
    def more_impotant_news(self, df, count_profession):
        try:
            return( df[df[count_profession] >= 4].sort_values(["date", count_profession], ascending = False, inplace = False))
        except:
            return( df[df[count_profession] >= 4].sort_values([count_profession], ascending = False, inplace = False))
        
        
        

In [8]:

#Создание объекта класса

preprocessor = DataPreprocessor()

# Создание основного дата сета
prime_news = preprocessor.prime_data_loading("TestwithData.csv")
#Добавление профессии бухгалтера, #Поиск тегов
preprocessor.prime_fit(prime_news, accountant)
#Очистка, кол.тегов
preprocessor.transform(prime_news, accountant)
#Добавление профессии бизнесменб #Поиск тегов
preprocessor.prime_fit(prime_news, business)
#Очистка, кол.тегов
preprocessor.transform(prime_news, business)

  0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

,links,title,text,date,union_im,tags_profession1,count_profession1,tags_profession2,count_profession2
0,http://www.consultant.ru//legalnews/20528/,Обзор новых антикризисных мер за 1 – 7 октября,См. материалы о частичной мобилизации и новых ...,2022-10-08,смотреть материал частичный мобилизация новый ...,новый юрлицо,2,новый юрлицо,2
1,http://www.consultant.ru//legalnews/20541/,О приостановке и возобновлении трудового догов...,Сведения о приостановке и возобновлении трудов...,2022-10-07,сведение приостановка возобновление трудовой д...,None,0,None,0
2,http://www.consultant.ru//legalnews/20505/,В срок не подтвердили деятельность – суды отме...,Если страхователь вовремя не подтвердит деятел...,2022-10-07,страхователь вовремя подтвердить деятельность ...,None,0,None,0
3,http://www.consultant.ru//legalnews/20538/,Учреждения по-прежнему должны передавать Казна...,Казначейство временно не размещает на сайте ww...,2022-10-07,казначейство временно размещать сайт план фхд ...,отчёт имущество,2,None,0
4,http://www.consultant.ru//legalnews/20492/,Суд указал на неосновательное обогащение: в до...,Организации установили в договоре цену с учето...,2022-10-07,организация установить договор цена учёт ндс о...,оплата налог,2,None,0
...,...,...,...,...,...,...,...,...,...
395,http://www.consultant.ru//legalnews/20122/,Не нужно вносить платежи по выкупному лизингу ...,Компания передала предприятию ТС по договору в...,2022-08-23,компания передать предприятие тс договор выкуп...,долг аванс финансирование имущество новый,5,финансирование новый,2
396,http://www.consultant.ru//legalnews/20124/,"УФАС напомнило, как отклонять заявку, которая ...",В закупке установили ограничения допуска иност...,2022-08-23,закупка установить ограничение допуск иностран...,None,0,товар,1
397,http://www.consultant.ru//legalnews/20123/,Работодатели вправе получить субсидию при найм...,С 23 августа программу господдержки при трудоу...,2022-08-23,август программа господдержка трудоустройство ...,работодатель,1,None,0
398,http://www.consultant.ru//legalnews/20120/,Утвердили стандарт медпомощи детям при железод...,С 29 августа лечение и диспансерное наблюдение...,2022-08-23,август лечение диспансерный наблюдение ребёнок...,None,0,None,0


In [9]:
# new_news = preprocessor.new_data_loading("new_news_dataset.csv")
# #Поиск тегов
# preprocessor.fit(new_news, accountant, 1)
# #Очистка кол.тегов
# preprocessor.transform(new_news, accountant)
# #Добавление профессии бизнесмен
# preprocessor.fit(new_news, business, 2)
# #Поиск тегов
# preprocessor.transform(new_news, business)


In [10]:
# #Добавление нового датафрейма к основному
# prime_news = preprocessor.data_concat(prime_news, new_news)

In [11]:
# Самые важные и свежие новости для бухгалтера
news_for_accountant = preprocessor.more_impotant_news(prime_news, "count_profession1")
news_for_accountant.drop(["tags_profession2", "count_profession2"], axis = 1, inplace = True)
news_for_accountant


,links,title,text,date,union_im,tags_profession1,count_profession1
43,http://www.consultant.ru//legalnews/20498/,Подборка интересных судебных споров за III ква...,"Краснодарский краевой суд решил, что врио руко...",2022-10-04,краснодарский краевой суд решить врио руководи...,зарплата аванс оплата недвижимость бухгалтер,5
65,http://www.consultant.ru//legalnews/20469/,Важные новости для бухгалтера за сентябрь,См. материалы о частичной мобилизации и новых ...,2022-10-03,смотреть материал частичный мобилизация новый ...,новый отчёт налог имущество изменение финансы ...,10
67,http://www.consultant.ru//legalnews/20457/,Важные новости для специалиста по кадрам за се...,См. материалы о частичной мобилизации и новых ...,2022-10-03,смотреть материал частичный мобилизация новый ...,новый работодатель зарплата оплата,4
71,http://www.consultant.ru//legalnews/20471/,Расчеты и налоги в октябре: на что обратить вн...,1. Электронные счета-фактуры по госконтрактам ...,2022-10-01,электронный счёт фактура госконтракт октябрь в...,новый изменение акт налог минфин отчёт,6
89,http://www.consultant.ru//legalnews/20447/,Что важного ждет бухгалтера в IV квартале,С 1 октября действует новая форма справки о со...,2022-09-29,октябрь действовать новый форма справка состоя...,новый справка налог изменение оплата кредитова...,9
92,http://www.consultant.ru//legalnews/20431/,Частичная мобилизация: обзор важных новостей д...,Последние измененияС 7 октября мобилизованные ...,2022-09-29,последний измененияс октябрь мобилизовать семь...,изменение отчёт зарплата новый акт налог,6
94,http://www.consultant.ru//legalnews/20434/,Какие важные изменения ждут юриста в IV кварта...,ЦБ РФ установил форму таблицы условий договора...,2022-09-29,цб рф установить форма таблица условие договор...,физлицо банкротство отчёт юрлицо имущество изм...,6
85,http://www.consultant.ru//legalnews/20451/,Единый налоговый платеж: предложили новые прав...,Минфин разместил для общественного обсуждения ...,2022-09-29,минфин разместить общественный обсуждение прое...,новый изменение налог минфин,4
95,http://www.consultant.ru//legalnews/20430/,Проект об оплате периода незанятости творчески...,"В ТК РФ планируют закрепить, что время, когда ...",2022-09-29,тк рф планировать закрепить время творческий р...,акт работодатель оплата изменение,4
133,http://www.consultant.ru//legalnews/20393/,Важные новости для бухгалтера за неделю с 19 п...,См. материал о новых антикризисных мерах 2022 ...,2022-09-26,смотреть материал новый антикризисный мера год...,новый налог изменение акт бухгалтер,5


In [12]:
# Самые важные и свежие новости для бизнеса
news_for_business = preprocessor.more_impotant_news(prime_news, "count_profession2")
news_for_business.drop(["tags_profession1", "count_profession1"], axis = 1, inplace = True)
news_for_business


,links,title,text,date,union_im,tags_profession2,count_profession2
62,http://www.consultant.ru//legalnews/20466/,Самые важные новости для юриста за сентябрь,См. материалы о частичной мобилизации и новых ...,2022-10-03,смотреть материал частичный мобилизация новый ...,новый физлицо товар кредит,4
89,http://www.consultant.ru//legalnews/20447/,Что важного ждет бухгалтера в IV квартале,С 1 октября действует новая форма справки о со...,2022-09-29,октябрь действовать новый форма справка состоя...,новый бизнес кредит бизнеса кредитование,5
94,http://www.consultant.ru//legalnews/20434/,Какие важные изменения ждут юриста в IV кварта...,ЦБ РФ установил форму таблицы условий договора...,2022-09-29,цб рф установить форма таблица условие договор...,физлицо банкротство товар юрлицо рынок,5
92,http://www.consultant.ru//legalnews/20431/,Частичная мобилизация: обзор важных новостей д...,Последние измененияС 7 октября мобилизованные ...,2022-09-29,последний измененияс октябрь мобилизовать семь...,бизнес новый товар управление,4
132,http://www.consultant.ru//legalnews/20388/,Важные новости для юриста за неделю с 19 по 23...,См. материал о новых антикризисных мерах 2022 ...,2022-09-26,смотреть материал новый антикризисный мера год...,новый кредит физлицо банкрот,4
141,http://www.consultant.ru//legalnews/20392/,Обзор новых антикризисных мер за 17 – 23 сентября,Информацию обо всех важнейших решениях по стаб...,2022-09-24,информация весь важный решение стабилизация эк...,бизнес новый физлицо кредит товар,5
150,http://www.consultant.ru//legalnews/20381/,Малому и среднему бизнесу рассказали об основн...,Кредитные каникулы – это отсрочка платежей на ...,2022-09-23,кредитный каникулы отсрочка платёж льготный пе...,кредит физлицо доход бизнес кредитование финан...,10
190,http://www.consultant.ru//legalnews/20329/,Важные новости для юриста за неделю с 12 по 16...,См. материал о новых антикризисных мерах 2022 ...,2022-09-19,смотреть материал новый антикризисный мера год...,новый физлицо клиент юрлицо,4
332,http://www.consultant.ru//legalnews/20193/,Важные новости для специалиста организации здр...,Минздрав выпустил новые временные рекомендации...,2022-09-01,минздрав выпустить новый временной рекомендаци...,новый юрлицо импорт кредит,4
356,http://www.consultant.ru//legalnews/20158/,Самые важные события для бухгалтера за прошедш...,См. материал о новых антикризисных мерах 2022 ...,2022-08-29,смотреть материал новый антикризисный мера год...,новый финансирование продукт доход товар рекла...,225


In [13]:
def tagged_document(list_of_ListOfWords):
    for x, ListOfWords in enumerate(list_of_ListOfWords):
        yield TaggedDocument(ListOfWords, [x])
data_train = list(tagged_document(prime_news.text))

d2v_model = Doc2Vec(vector_size=100, min_count=2, epochs=50)
d2v_model.build_vocab(data_train)
d2v_model.train(data_train, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

In [14]:
def dubles(df, model):
        doubles = []
        def distCosine (vecA, vecB):
            def dotProduct (vecA, vecB):
                d = 0.0
                for i in range(len(vecA)):
                    d += vecA[i] * vecB[i]
                return d
            return dotProduct (vecA,vecB) / math.sqrt(dotProduct(vecA,vecA)) / math.sqrt(dotProduct(vecB,vecB))
        lens = 50
        eps = 0.0005
        while lens > 10:
            corp = df.iloc[0:50]
            corp = corp.reset_index()
            for i in range(len(corp) - 1, -1, -1):
                emb0 = model.infer_vector(list(corp.iloc[i].text.split()))
                for j in range(i - 1, -1, -1):
                    emb1 = model.infer_vector(list(corp.iloc[j].text.split()))
                    if abs(distCosine(emb0, emb1)) < eps:
                        doubles.append([corp.iloc[i].links, corp.iloc[j].links])
                        corp = corp.iloc[0 : i]
                        break
            lens = len(corp)
            eps += 0.0003
        return [corp, doubles]

In [15]:
news_for_accountant = dubles(news_for_accountant,d2v_model)[0]
news_for_business = dubles(news_for_business,d2v_model)[0]

In [16]:
def topics(df):
    ans = []
    tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df = 0)
    tfidf_matrix =  tfidf.fit_transform(df.text)
    feature_names = tfidf.get_feature_names() 
    dense = tfidf_matrix.todense()
    for i in range(len(df)):
        text = dense[i].tolist()[0]
        phrase_scores = [pair for pair in zip(range(0, len(text)), text) if pair[1] > 0]
        sorted_phrase_scores = sorted(phrase_scores, key=lambda t: t[1] * -1)
        tfidf_ranking = []
        for phrase, score in [(feature_names[word_id], score) for (word_id, score) in sorted_phrase_scores][:6]:
            tfidf_ranking.append(phrase)
        ans.append(tfidf_ranking)
    return ans

def main_key_words(topic):
    ans = {}
    for i in range(len(topic)):
        for j in range(len(topic[i])):
            word = topic[i][j]
            if word in ans:
                ans[word] += 1
            else:
                ans[word] = 1
    sorted_dictionary = sorted(ans.items() , key=lambda t : t[1] , reverse=True)
    ans = {}
    for el in sorted_dictionary[2:10]:
        ans[el[0]] = el[1]
    return ans

def trendswords(ctop, ptop):
    for key, value in ptop.items():
        if key in ctop:
            diff = ctop[key] - ptop[key]
            if diff > 0:
                ctop[key] = diff
            else:
                del ctop[key]
    sorted_dictionary = sorted(ctop.items() , key=lambda t : t[1] , reverse=True)
    ans = {}
    for el in sorted_dictionary:
        ans[el[0]] = el[1]
    return ans

def trendout(df, twords):
    ans = {}
    keywords = topics(df)
    for i in range(len(keywords)):
        ball = 0
        for key, value in twords.items():
            if key in keywords[i]:
                ball += int(value)
        ans[i] = ball
    sorted_dictionary = sorted(ans.items() , key=lambda t : t[1] , reverse=True)[0:5]
    links = []
    for el in sorted_dictionary:
        links.append(df.iloc[int(el[0])].links)
    return links

def trendlinks(df):
    current_week_filter = df.date.max() - pd.Timedelta(7, "d")
    previous_week_filter = current_week_filter - pd.Timedelta(7, "d")
    currentw_data = prime_news[prime_news.date > current_week_filter]
    previousw_data = prime_news[(prime_news.date < current_week_filter) & (prime_news.date > previous_week_filter)]
    currentw_topics = topics(currentw_data)
    previousw_topics = topics(previousw_data)
    currentw_keywords = main_key_words(currentw_topics)
    previousw_keywords = main_key_words(previousw_topics)
    finallinks = trendout(currentw_data, trendswords(currentw_keywords, previousw_keywords))
    return finallinks

trendlinks2 = trendlinks(prime_news)

In [17]:
bs = news_for_accountant.head(3)
ac = news_for_business.head(3)
bs["role"] = 'business'
ac["role"] = 'accountant'
bsac2 = pd.concat([bs, ac], axis=0, ignore_index=True)
bsac2.to_csv("concatArtur.csv", sep=",", index=False)


In [18]:
trendlinks2 = pd.Series(trendlinks2)
trendlinks2.to_pickle("trendlinks")

In [19]:
bsac2

,index,links,title,text,date,union_im,tags_profession1,count_profession1,role,tags_profession2,count_profession2
0,43,http://www.consultant.ru//legalnews/20498/,Подборка интересных судебных споров за III ква...,"Краснодарский краевой суд решил, что врио руко...",2022-10-04,краснодарский краевой суд решить врио руководи...,зарплата аванс оплата недвижимость бухгалтер,5.0,business,NaN,NaN
1,65,http://www.consultant.ru//legalnews/20469/,Важные новости для бухгалтера за сентябрь,См. материалы о частичной мобилизации и новых ...,2022-10-03,смотреть материал частичный мобилизация новый ...,новый отчёт налог имущество изменение финансы ...,10.0,business,NaN,NaN
2,67,http://www.consultant.ru//legalnews/20457/,Важные новости для специалиста по кадрам за се...,См. материалы о частичной мобилизации и новых ...,2022-10-03,смотреть материал частичный мобилизация новый ...,новый работодатель зарплата оплата,4.0,business,NaN,NaN
3,62,http://www.consultant.ru//legalnews/20466/,Самые важные новости для юриста за сентябрь,См. материалы о частичной мобилизации и новых ...,2022-10-03,смотреть материал частичный мобилизация новый ...,NaN,NaN,accountant,новый физлицо товар кредит,4.0
4,89,http://www.consultant.ru//legalnews/20447/,Что важного ждет бухгалтера в IV квартале,С 1 октября действует новая форма справки о со...,2022-09-29,октябрь действовать новый форма справка состоя...,NaN,NaN,accountant,новый бизнес кредит бизнеса кредитование,5.0
5,94,http://www.consultant.ru//legalnews/20434/,Какие важные изменения ждут юриста в IV кварта...,ЦБ РФ установил форму таблицы условий договора...,2022-09-29,цб рф установить форма таблица условие договор...,NaN,NaN,accountant,физлицо банкротство товар юрлицо рынок,5.0


In [20]:
trendlinks2

0    http://www.consultant.ru//legalnews/20498/
1    http://www.consultant.ru//legalnews/20473/
2    http://www.consultant.ru//legalnews/20536/
3    http://www.consultant.ru//legalnews/20516/
4    http://www.consultant.ru//legalnews/20503/
dtype: object